In [1]:
import os
os.chdir('/home/smallyan/eval_agent')
print(f"Working directory: {os.getcwd()}")

Working directory: /home/smallyan/eval_agent


# Generalizability Evaluation for Universal Neurons

This notebook evaluates whether the findings in the `universal-neurons_eval` repository generalize beyond the original experimental setting.

## Evaluation Checklist
- **GT1**: Generalization to a New Model
- **GT2**: Generalization to New Data  
- **GT3**: Method / Specificity Generalizability

## Setup

In [2]:
# Explore the repository structure
import subprocess

repo_path = '/net/scratch2/smallyan/universal-neurons_eval'
result = subprocess.run(['find', repo_path, '-type', 'f', '-name', '*.py'], capture_output=True, text=True)
print("Python files in the repository:")
print(result.stdout)

result2 = subprocess.run(['find', repo_path, '-type', 'f', '-name', '*.json'], capture_output=True, text=True)
print("\nJSON files in the repository:")
print(result2.stdout)

result3 = subprocess.run(['find', repo_path, '-type', 'f', '-name', '*.md'], capture_output=True, text=True)
print("\nMarkdown files in the repository:")
print(result3.stdout)

Python files in the repository:
/net/scratch2/smallyan/universal-neurons_eval/summary.py
/net/scratch2/smallyan/universal-neurons_eval/entropy_intervention.py
/net/scratch2/smallyan/universal-neurons_eval/activations.py
/net/scratch2/smallyan/universal-neurons_eval/make_dataset.py
/net/scratch2/smallyan/universal-neurons_eval/attention_deactivation_qpos.py
/net/scratch2/smallyan/universal-neurons_eval/correlations_parallel.py
/net/scratch2/smallyan/universal-neurons_eval/summary_viewer.py
/net/scratch2/smallyan/universal-neurons_eval/weights.py
/net/scratch2/smallyan/universal-neurons_eval/attention_deactivation.py
/net/scratch2/smallyan/universal-neurons_eval/correlations_fast.py
/net/scratch2/smallyan/universal-neurons_eval/explain.py
/net/scratch2/smallyan/universal-neurons_eval/utils.py
/net/scratch2/smallyan/universal-neurons_eval/correlations.py
/net/scratch2/smallyan/universal-neurons_eval/analysis/weights.py
/net/scratch2/smallyan/universal-neurons_eval/analysis/vocab_df.py
/ne

In [3]:
# Read the plan file to understand the research
with open(f'{repo_path}/plan.md', 'r') as f:
    plan_content = f.read()
print(plan_content[:5000])

# Plan
## Objective
Study the universality of individual neurons across GPT2 language models trained from different random seeds to identify interpretable neurons and understand whether neural mechanisms are universal across models.

## Hypothesis
1. Universal neurons (those that consistently activate on the same inputs across different models) are more likely to be monosemantic and interpretable than non-universal neurons.
2. Neurons with high activation correlation across models will have clear interpretations and can be taxonomized into a small number of neuron families.
3. Universal neurons exhibit specific statistical properties in their weights and activations that distinguish them from non-universal neurons, including large negative input bias, high pre-activation skew and kurtosis, and large weight norm.

## Methodology
1. Compute pairwise Pearson correlations of neuron activations over 100 million tokens from the Pile test set for every neuron pair across five GPT2 models trai

In [4]:
# Continue reading the plan
print(plan_content[5000:])

ing. Anti-entropy neuron (L22.2882) has opposite effect with cos similarity -0.886 to entropy neuron.

### Attention head deactivation neurons via path ablation
- What varied: Path ablation of neuron L4.3594 on attention head L5.H0; analyzed all neuron-head pairs using heuristic score hn=WTout*WTQ*kBOS
- Metric: Change in BOS attention and head output norm when neuron contribution is ablated; heuristic score distribution vs random baseline
- Main result: Heuristic identifies neurons controlling BOS attention. Neuron L4.3594 increases BOS attention and decreases head L5.H0 output norm when activated (deactivation neuron). Median head has WO*vBOS norm 19.4x smaller than other tokens, enabling heads to turn off by attending to BOS.


In [5]:
# Read the CodeWalkthrough to understand the implementation
with open(f'{repo_path}/CodeWalkthrough.md', 'r') as f:
    walkthrough = f.read()
print(walkthrough[:5000])

# Universal Neurons
All supporting data and code for Universal Neurons in GPT2 Language Models by Gurnee et al. (2024).

## Contents
* `dataframes/neuron_dfs` contains dataframes with neuron statistics for all neurons for the main models studies.
* `paper_notebooks` contains much of the plotting code to generate the figures in the paper.
* `correlations_fast.py` contains the script to compute neuron correlations.
* `summary.py` and `weights.py` contain scripts to compute neuron activation and weight statistic summaries for use of our summary viewer (contained in `summary_viewer.py`). See next section for more information on the data generated.
* `activations.py` contains scripts to cache neuron activations.
* `explain.py` contains script to compute our reduction in variance explanations.
* `attention_deactivation.py`, `entropy_intervention.py`, and `intervention.py` contain scripts for our functional neuron experiments.
* The `analysis` directory contains further plotting and analysis 

In [6]:
# Check available data in the repository
import os

# Check directories
dirs_to_check = ['dataframes', 'paper_notebooks', 'summary_data', 'evaluation']
for d in dirs_to_check:
    full_path = os.path.join(repo_path, d)
    if os.path.exists(full_path):
        print(f"\n{d}/ exists:")
        try:
            contents = os.listdir(full_path)[:20]
            for c in contents:
                print(f"  - {c}")
        except:
            print("  Could not list contents")
    else:
        print(f"\n{d}/ does not exist")


dataframes/ exists:
  - interpretable_neurons
  - neuron_dfs
  - vocab_dfs

paper_notebooks/ exists:
  - previous_token_neurons.ipynb
  - bos_signal_neurons.ipynb
  - properties_of_universal_neurons.ipynb
  - alphabet_neurons.ipynb
  - unigram_neurons.ipynb
  - prediction_neurons.ipynb
  - syntax_neurons.ipynb
  - position_neurons.ipynb
  - entropy_neurons.ipynb
  - mysteries.ipynb
  - topic_neurons.ipynb
  - family_count.ipynb

summary_data/ does not exist

evaluation/ exists:
  - replications
  - self_matching.ipynb
  - consistency_evaluation.json


In [7]:
# Check the neuron dataframes to understand what neurons were identified
import pandas as pd

neuron_df_path = os.path.join(repo_path, 'dataframes', 'neuron_dfs')
contents = os.listdir(neuron_df_path)
print("Neuron dataframes available:")
for c in contents:
    print(f"  - {c}")

Neuron dataframes available:
  - stanford-gpt2-small-a.csv
  - pythia-160m.csv
  - stanford-gpt2-medium-a.csv


In [8]:
# Load and examine the neuron dataframes to understand the findings
df_gpt2_medium = pd.read_csv(os.path.join(neuron_df_path, 'stanford-gpt2-medium-a.csv'))
print("GPT2-medium-a neuron dataframe:")
print(f"Shape: {df_gpt2_medium.shape}")
print(f"Columns: {df_gpt2_medium.columns.tolist()}")
print("\nFirst few rows:")
df_gpt2_medium.head()

GPT2-medium-a neuron dataframe:
Shape: (98304, 22)
Columns: ['layer', 'neuron', 'max_corr', 'mean_corr', 'min_corr', 'max_baseline', 'min_baseline', 'mean_baseline', 'w_in_norm', 'input_bias', 'w_out_norm', 'in_out_sim', 'l2_penalty', 'mean', 'var', 'skew', 'kurt', 'vocab_mean', 'vocab_var', 'vocab_skew', 'vocab_kurt', 'sparsity']

First few rows:


,layer,neuron,max_corr,mean_corr,min_corr,max_baseline,min_baseline,mean_baseline,w_in_norm,input_bias,...,l2_penalty,mean,var,skew,kurt,vocab_mean,vocab_var,vocab_skew,vocab_kurt,sparsity
0,0,0,0.4646,0.439825,0.4092,0.3500,0.3330,0.342250,0.370499,-0.388558,...,0.271325,-1.077630,0.410425,0.063839,2.780662,0.000301,0.000995,0.007442,2.978626,0.057869
1,0,1,0.2600,0.228850,0.2119,0.1885,0.1608,0.168700,0.368054,-0.367035,...,0.251356,-0.619509,0.104088,0.417814,4.443205,-0.000187,0.000939,0.017759,2.983132,0.057327
2,0,2,0.5570,0.518625,0.4575,0.1963,0.1887,0.190875,0.395692,-0.539252,...,0.288211,-1.267382,0.312051,0.506417,3.808384,-0.000063,0.000957,-0.001548,3.005584,0.028301
3,0,3,0.3880,0.340350,0.3157,0.2527,0.2285,0.240050,0.392904,-0.490780,...,0.280817,-1.150307,0.335263,0.261270,3.022481,-0.000015,0.000930,0.014691,2.992840,0.037805
4,0,4,0.2878,0.263300,0.2396,0.1550,0.1460,0.150250,0.385215,-0.417577,...,0.262087,-0.711964,0.165003,0.494364,4.288840,0.000254,0.000920,-0.004821,3.000447,0.061776


In [9]:
# Find the universal neurons (excess correlation > 0.5)
# Excess correlation = max_corr - max_baseline
df_gpt2_medium['excess_corr'] = df_gpt2_medium['max_corr'] - df_gpt2_medium['max_baseline']
universal_neurons = df_gpt2_medium[df_gpt2_medium['excess_corr'] > 0.5]
print(f"Number of universal neurons in GPT2-medium-a: {len(universal_neurons)} / {len(df_gpt2_medium)} ({100*len(universal_neurons)/len(df_gpt2_medium):.2f}%)")
print("\nTop 20 universal neurons by excess correlation:")
universal_neurons.sort_values('excess_corr', ascending=False).head(20)[['layer', 'neuron', 'max_corr', 'max_baseline', 'excess_corr']]

Number of universal neurons in GPT2-medium-a: 2010 / 98304 (2.04%)

Top 20 universal neurons by excess correlation:


,layer,neuron,max_corr,max_baseline,excess_corr
4753,1,657,0.9360,0.12520,0.81080
5127,1,1031,0.9550,0.14950,0.80550
185,0,185,0.9670,0.16410,0.80290
553,0,553,0.9805,0.17770,0.80280
5489,1,1393,0.9450,0.15670,0.78830
93509,22,3397,0.9277,0.13950,0.78820
13640,3,1352,0.9653,0.17970,0.78560
5662,1,1566,0.9450,0.16050,0.78450
6877,1,2781,0.9710,0.18840,0.78260
345,0,345,0.9673,0.18540,0.78190


In [10]:
# Check the interpretable neurons folder
interp_path = os.path.join(repo_path, 'dataframes', 'interpretable_neurons')
if os.path.exists(interp_path):
    print("Interpretable neurons files:")
    for f in os.listdir(interp_path):
        print(f"  - {f}")
        full_file = os.path.join(interp_path, f)
        if f.endswith('.csv'):
            df = pd.read_csv(full_file)
            print(f"    Shape: {df.shape}")
            print(f"    Columns: {df.columns.tolist()[:10]}")
else:
    print("No interpretable_neurons directory found")

Interpretable neurons files:
  - stanford-gpt2-medium-a
  - stanford-gpt2-small-a
  - pythia-160m


In [11]:
# Check the contents of one of the interpretable neurons folders
interp_gpt2_medium = os.path.join(interp_path, 'stanford-gpt2-medium-a')
if os.path.exists(interp_gpt2_medium):
    print("Contents of stanford-gpt2-medium-a:")
    for f in os.listdir(interp_gpt2_medium):
        print(f"  - {f}")

Contents of stanford-gpt2-medium-a:
  - universal.csv
  - prediction_neurons.csv


In [12]:
# Load the universal neurons file
universal_neurons_df = pd.read_csv(os.path.join(interp_gpt2_medium, 'universal.csv'))
print(f"Universal neurons CSV shape: {universal_neurons_df.shape}")
print(f"Columns: {universal_neurons_df.columns.tolist()}")
print("\nFirst 10 rows:")
universal_neurons_df.head(10)

Universal neurons CSV shape: (1211, 23)
Columns: ['layer', 'neuron', 'max_corr', 'mean_corr', 'min_corr', 'max_baseline', 'min_baseline', 'mean_baseline', 'w_in_norm', 'input_bias', 'w_out_norm', 'in_out_sim', 'l2_penalty', 'mean', 'var', 'skew', 'kurt', 'vocab_mean', 'vocab_var', 'vocab_skew', 'vocab_kurt', 'sparsity', 'excess_corr']

First 10 rows:


,layer,neuron,max_corr,mean_corr,min_corr,max_baseline,min_baseline,mean_baseline,w_in_norm,input_bias,...,mean,var,skew,kurt,vocab_mean,vocab_var,vocab_skew,vocab_kurt,sparsity,excess_corr
0,0,37,0.7236,0.702625,0.6772,0.1907,0.1800,0.185625,0.387450,-0.447526,...,-1.103980,0.316467,0.510973,3.621278,0.000199,0.000977,-0.016318,2.996915,0.041241,0.517000
1,0,45,0.7637,0.732850,0.6943,0.2158,0.1890,0.198325,0.376307,-0.409769,...,-0.912744,0.254701,0.718533,4.532629,-0.000166,0.000980,-0.001359,3.019827,0.056931,0.534525
2,0,50,0.7886,0.734850,0.6973,0.1810,0.1556,0.169125,0.356303,-0.392182,...,-0.660341,0.165108,1.585033,9.291728,0.000309,0.000947,-0.010772,2.990849,0.076345,0.565725
3,0,135,0.9624,0.957100,0.9510,0.3694,0.2732,0.313525,0.348459,-0.345287,...,-0.698817,0.225584,2.267953,12.477121,0.000074,0.000952,-0.055805,2.993432,0.070599,0.643575
4,0,185,0.9670,0.937175,0.9194,0.1641,0.1492,0.155200,0.405053,-0.610551,...,-0.987586,0.267893,1.985424,21.138607,0.000645,0.000988,0.100476,3.083210,0.020191,0.781975
5,0,196,0.7680,0.717825,0.6826,0.2108,0.1847,0.199900,0.338234,-0.318337,...,-0.500606,0.183664,2.847586,16.874023,0.000486,0.001069,0.074644,3.125614,0.083778,0.517925
6,0,197,0.9450,0.943300,0.9414,0.3223,0.2686,0.294575,0.349206,-0.361166,...,-0.514652,0.233543,2.905099,17.004550,0.000196,0.000956,0.021488,3.009044,0.101384,0.648725
7,0,205,0.8850,0.864850,0.8390,0.1833,0.1592,0.169450,0.391586,-0.405299,...,-0.771277,0.219925,1.266032,11.711295,-0.000276,0.000948,0.027749,3.075940,0.057483,0.695400
8,0,345,0.9673,0.898925,0.7954,0.1854,0.1544,0.168900,0.377022,-0.703568,...,-1.405397,0.556662,0.493350,5.038054,-0.001262,0.001196,0.286076,3.305293,0.029009,0.730025
9,0,348,0.9087,0.826350,0.6920,0.1803,0.1713,0.175575,0.373433,-0.349906,...,-0.791489,0.253866,1.257878,11.766571,-0.000491,0.000906,0.018181,3.050753,0.067014,0.650775


In [13]:
# Check prediction neurons
prediction_neurons_df = pd.read_csv(os.path.join(interp_gpt2_medium, 'prediction_neurons.csv'))
print(f"Prediction neurons CSV shape: {prediction_neurons_df.shape}")
print(f"Columns: {prediction_neurons_df.columns.tolist()}")
print("\nFirst 10 rows:")
prediction_neurons_df.head(10)

Prediction neurons CSV shape: (136, 4)
Columns: ['layer', 'neuron', 'feature', 'is_surpress']

First 10 rows:


,layer,neuron,feature,is_surpress
0,20,13,all_caps,False
1,23,3440,all_caps,True
2,21,2148,all_caps,True
3,15,591,all_caps,True
4,19,1121,all_caps,False
5,18,2336,all_caps,False
6,15,84,all_caps,False
7,17,2559,all_caps,False
8,22,1585,end_w_ing,False
9,22,3534,end_w_ing,False


In [14]:
# Check what features are tracked
print("Unique features in prediction neurons:")
print(prediction_neurons_df['feature'].unique())
print("\nFeature counts:")
prediction_neurons_df['feature'].value_counts()

Unique features in prediction neurons:
['all_caps' 'end_w_ing' 'is_year' 'is_intensive_pronoun'
 'is_second_person_pronoun' 'is_female_pronoun' 'is_neutral_pronoun'
 'is_male_pronoun' 'contains_question' 'is_one_digit'
 'contains_exclamation' 'is_relative_pronoun' 'start_w_no_space_and_digit'
 'contains_close_paren' 'contains_quotation' 'is_interrogative_pronoun'
 'is_month' 'contains_semicolon' 'contains_open_bracket'
 'contains_open_paren' 'is_demonstrative_pronoun' 'contains_digit'
 'contains_colon' 'is_state' 'is_possessive_pronoun']

Feature counts:


feature
end_w_ing                     15
is_neutral_pronoun            10
is_second_person_pronoun       9
all_caps                       8
contains_colon                 8
is_demonstrative_pronoun       8
contains_open_paren            8
is_year                        5
contains_digit                 5
is_male_pronoun                5
contains_question              5
is_relative_pronoun            5
contains_close_paren           5
contains_open_bracket          5
is_interrogative_pronoun       5
is_month                       4
start_w_no_space_and_digit     4
contains_quotation             4
is_possessive_pronoun          4
contains_semicolon             3
is_female_pronoun              3
is_state                       3
contains_exclamation           2
is_one_digit                   2
is_intensive_pronoun           1
Name: count, dtype: int64

In [15]:
# Look at the replications folder to understand what has been replicated
replications_path = os.path.join(repo_path, 'evaluation', 'replications')
if os.path.exists(replications_path):
    print("Replications contents:")
    for f in os.listdir(replications_path):
        print(f"  - {f}")
        subpath = os.path.join(replications_path, f)
        if os.path.isdir(subpath):
            for sf in os.listdir(subpath)[:10]:
                print(f"      - {sf}")

Replications contents:
  - universal_neurons_properties.png
  - layer_distribution.png


In [16]:
# Read the consistency evaluation file
import json
consistency_path = os.path.join(repo_path, 'evaluation', 'consistency_evaluation.json')
with open(consistency_path, 'r') as f:
    consistency_eval = json.load(f)
print("Consistency evaluation:")
print(json.dumps(consistency_eval, indent=2))

Consistency evaluation:
{
  "Checklist": {
    "CS1_Results_vs_Conclusion": "PASS",
    "CS2_Plan_vs_Implementation": "PASS"
  },
  "Rationale": {
    "CS1_Results_vs_Conclusion": "All evaluable conclusions in the plan match the recorded results in the implementation notebooks: (1) Universal neuron percentages match exactly - GPT2-medium 1.23%, Pythia-160M 1.26%, GPT2-small 4.16% as claimed in plan and verified in family_count.ipynb/neuron dataframes; (2) Statistical properties of universal neurons match - they have more negative input bias (-0.486 vs -0.248), higher activation skew (1.100 vs 0.024), and higher kurtosis (8.111 vs 3.735) as shown in properties_of_universal_neurons.ipynb; (3) Prediction neuron layer distribution matches - high vocab_kurt neurons are concentrated in later layers (17-23) with 1136 neurons after midpoint vs 125 before, confirming the plan claim that prediction neurons become prevalent after network midpoint.",
    "CS2_Plan_vs_Implementation": "A plan.md fi

In [17]:
# Check available GPU and load necessary libraries
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

CUDA available: True
CUDA device: NVIDIA A100 80GB PCIe
GPU Memory: 85.1 GB


## Summary of Repository Findings

Based on the exploration:

1. **Main Finding**: The paper identifies "universal neurons" - neurons that activate on the same inputs across different models trained from different random seeds. Universal neurons have excess correlation > 0.5 and comprise only 1-5% of neurons.

2. **Key Properties of Universal Neurons**:
   - Large negative input bias
   - High activation skew and kurtosis (monosemantic signature)
   - High weight norm
   - Lower activation frequency

3. **Neuron Families Identified**:
   - Unigram neurons (specific tokens)
   - Alphabet neurons
   - Previous token neurons
   - Position neurons
   - Syntax neurons
   - Semantic/context neurons
   - Prediction neurons (high vocab_kurt)
   - Entropy neurons

4. **Models Used in Original Work**:
   - GPT2-small (stanford-gpt2-small-a)
   - GPT2-medium (stanford-gpt2-medium-a)
   - Pythia-160m

5. **Dataset Used**: Pile test set (100 million tokens)

---

## GT1: Generalization to a New Model

We will test whether the universal neuron findings generalize to a model NOT used in the original work.

**New model to test**: GPT2-large or a different Pythia model (e.g., Pythia-410m)

In [18]:
# Load environment variables for API access
import subprocess
result = subprocess.run(['bash', '-c', 'source /home/smallyan/.bashrc && env'], capture_output=True, text=True)
env_vars = {}
for line in result.stdout.split('\n'):
    if '=' in line:
        key, _, value = line.partition('=')
        env_vars[key] = value
        if 'TOKEN' in key or 'API' in key or 'KEY' in key:
            os.environ[key] = value
            print(f"Set {key}=****")
            
# Also try sourcing directly
import os
hf_token_path = os.path.expanduser('~/.bashrc')
print("\nChecking for HF token...")

Set OPENAI_API_KEY=****
Set HF_TOKEN=****
Set NDIF_API_KEY=****
Set OPENROUTER_API_KEY=****

Checking for HF token...


In [19]:
# Install required packages and load models
import sys
!{sys.executable} -m pip install transformers datasets torch numpy scipy -q

In [20]:
# Load the original model (GPT2-medium) and a new model (GPT2-large - not used in original paper)
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoModelForCausalLM, AutoTokenizer
import numpy as np
from scipy import stats

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load GPT2-medium (from original study)
print("Loading GPT2-medium...")
gpt2_medium = GPT2LMHeadModel.from_pretrained('gpt2-medium').to(device)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

# Load GPT2-large (NOT used in original study - for GT1 test)
print("Loading GPT2-large...")
gpt2_large = GPT2LMHeadModel.from_pretrained('gpt2-large').to(device)
tokenizer_large = GPT2Tokenizer.from_pretrained('gpt2-large')

print("Models loaded successfully!")

Using device: cuda
Loading GPT2-medium...


OSError: PermissionError at /net/projects2/chai-lab/shared_models/hub/.locks/models--gpt2-medium/fc5a354a19255ad494f3d71549390baca1ccf61d1d822b9408971705c687c9cd.lock when downloading gpt2-medium. Check cache directory permissions. Common causes: 1) another user is downloading the same model (please wait); 2) a previous download was canceled and the lock file needs manual removal.

In [21]:
# Set a custom cache directory to avoid permission issues
import os
os.environ['HF_HOME'] = '/home/smallyan/.cache/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/home/smallyan/.cache/huggingface/transformers'
os.makedirs('/home/smallyan/.cache/huggingface/transformers', exist_ok=True)

# Reload transformers
import importlib
import transformers
importlib.reload(transformers)

from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load GPT2-medium (from original study)
print("Loading GPT2-medium...")
gpt2_medium = GPT2LMHeadModel.from_pretrained('gpt2-medium', cache_dir='/home/smallyan/.cache/huggingface/transformers').to(device)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', cache_dir='/home/smallyan/.cache/huggingface/transformers')

print("GPT2-medium loaded successfully!")

Using device: cuda
Loading GPT2-medium...


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [22]:
# Check if model was loaded
print(f"GPT2-medium loaded: {gpt2_medium is not None}")
print(f"Model config: {gpt2_medium.config.n_layer} layers, {gpt2_medium.config.n_head} heads, {gpt2_medium.config.n_embd} embd dim")

In [23]:
# Wait and retry to check model
import time
time.sleep(5)
try:
    print(f"GPT2-medium loaded: {gpt2_medium is not None}")
    print(f"Model config: {gpt2_medium.config.n_layer} layers, {gpt2_medium.config.n_head} heads, {gpt2_medium.config.n_embd} embd dim")
except Exception as e:
    print(f"Error: {e}")

In [24]:
# The model download might be running. Let me check if it completed
print("Checking model status...")
print(type(gpt2_medium))

In [25]:
# Simple print test
print("Test output")